In [2]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [4]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [14]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 594 ms, sys: 8.42 ms, total: 603 ms
Wall time: 601 ms


In [18]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 373 ms, sys: 7.6 ms, total: 380 ms
Wall time: 401 ms


In [20]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 159 ms, sys: 8.71 ms, total: 168 ms
Wall time: 168 ms


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

In [22]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 936 μs, sys: 300 μs, total: 1.24 ms
Wall time: 1.08 ms


0       -38.114105
1       -38.125487
2       -38.084228
3       -38.065981
4       -38.045047
           ...    
27926   -38.106420
27927   -38.113307
27928   -38.121112
27929   -38.108484
27930   -38.151707
Length: 27931, dtype: float64

# Practice

In [24]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [26]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 279     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         1308   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         4675        DAY  OTHERS         THEFT F/AUTO   
         11621   EVENING  OTHERS          THEFT/OTHER   
         15616       DAY  OTHERS          THEFT/OTHER   

                                                     BLOCK    XBLOCK  \
20165648 279            1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         1308             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         4675          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         11621  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15616   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 279    137478.0  ...       10603.0    Precinct 144  38.905159   
         1308   135959.0  ...        8200.0     Precinct 89  38.891475   
         4675   137456.0  ...       10602.0     Precinct 83  38.904961   
         11621  136899.0  ...        5900.0    Precinct 143  38.899942   
         15616  135255.0  ...        6500.0     Precinct 89  38.885133   

                LONGITUDE           BID              START_DATE  \
20165648 279   -77.005891          NOMA  2020/11/20 20:15:26+00   
         1308  -76.999516           NaN  2020/11/20 03:02:27+00   
         4675  -76.997314           NaN  2020/11/19 22:30:39+00   
         11621 -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         15616 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 279    2020/11/20 21:46:24+00  596093043             NaN   
         1308                      NaN  596105470             NaN   
         4675   2020/11/20 03:00:43+00  596362501             NaN   
         11621  2020/11/20 22:08:28+00  596387673             NaN   
         15616                     NaN  596402829             NaN   

                               report_dt  
20165648 279   2020-11-20 22:17:27+00:00  
         1308  2020-11-20 04:27:36+00:00  
         4675  2020-11-20 15:06:04+00:00  
         11621 2020-11-20 22:07:10+00:00  
         15616 2020-11-20 12:46:32+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [8]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 491   -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         10824 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14887 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         16277 -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         16280 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 491     EVENING  OTHERS          THEFT/OTHER   
         10824       DAY  OTHERS          THEFT/OTHER   
         14887   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16277  MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         16280       DAY  OTHERS         THEFT F/AUTO   

                                                     BLOCK    XBLOCK  \
20165648 491    300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         10824   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14887          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         16277            100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         16280              300 - 399 BLOCK OF G STREET NE  399886.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 491    136899.0  ...        5900.0    Precinct 143  38.899942   
         10824  135255.0  ...        6500.0     Precinct 89  38.885133   
         14887  137478.0  ...       10603.0    Precinct 144  38.905159   
         16277  135959.0  ...        8200.0     Precinct 89  38.891475   
         16280  136784.0  ...        8301.0     Precinct 83  38.898907   

                LONGITUDE           BID              START_DATE  \
20165648 491   -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         10824 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   
         14887 -77.005891          NOMA  2020/11/20 20:15:26+00   
         16277 -76.999516           NaN  2020/11/20 03:02:27+00   
         16280 -77.001314           NaN  2020/11/20 15:30:02+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 491    2020/11/20 22:08:28+00  499865401             NaN   
         10824                     NaN  500146815             NaN   
         14887  2020/11/20 21:46:24+00  500163163             NaN   
         16277                     NaN  500182525             NaN   
         16280  2020/11/20 18:25:35+00  500182528             NaN   

                               report_dt  
20165648 491   2020-11-20 22:07:10+00:00  
         10824 2020-11-20 12:46:32+00:00  
         14887 2020-11-20 22:17:27+00:00  
         16277 2020-11-20 04:27:36+00:00  
         16280 2020-11-20 18:56:18+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [32]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    return same_wards_sametime

### 2.2 apply it to one of the focal crimes

In [34]:
find_related_crimes(C_Tar.iloc[0])

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
279,-77.005894,38.905167,20165986,2020/11/20 22:17:27+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1151 - 1199 BLOCK OF 1ST STREET NE,399489.0,137478.0,...,10603.0,Precinct 144,38.905159,-77.005891,NOMA,2020/11/20 20:15:26+00,2020/11/20 21:46:24+00,596093043,NaN,2020-11-20 22:17:27+00:00
1308,-76.999518,38.891483,20165709,2020/11/20 04:27:36+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,100 - 199 BLOCK OF 5TH STREET NE,400042.0,135959.0,...,8200.0,Precinct 89,38.891475,-76.999516,NaN,2020/11/20 03:02:27+00,NaN,596105470,NaN,2020-11-20 04:27:36+00:00
4675,-76.997316,38.904969,20165805,2020/11/20 15:06:04+00,DAY,OTHERS,THEFT F/AUTO,600 - 699 BLOCK OF ORLEANS PLACE NE,400233.0,137456.0,...,10602.0,Precinct 83,38.904961,-76.997314,NaN,2020/11/19 22:30:39+00,2020/11/20 03:00:43+00,596362501,NaN,2020-11-20 15:06:04+00:00
11621,-77.015554,38.899950,20166039,2020/11/20 22:07:10+00,EVENING,OTHERS,THEFT/OTHER,300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW,398651.0,136899.0,...,5900.0,Precinct 143,38.899942,-77.015552,DOWNTOWN,2020/11/20 17:30:16+00,2020/11/20 22:08:28+00,596387673,NaN,2020-11-20 22:07:10+00:00
15616,-76.997328,38.885141,20165798,2020/11/20 12:46:32+00,DAY,OTHERS,THEFT/OTHER,600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE,400232.0,135255.0,...,6500.0,Precinct 89,38.885133,-76.997326,CAPITOL HILL,2020/11/19 23:43:15+00,NaN,596402829,NaN,2020-11-20 12:46:32+00:00
15617,-76.997316,38.904969,20165803,2020/11/20 14:45:06+00,DAY,OTHERS,THEFT F/AUTO,600 - 699 BLOCK OF ORLEANS PLACE NE,400233.0,137456.0,...,10602.0,Precinct 83,38.904961,-76.997314,NaN,2020/11/19 23:45:48+00,2020/11/20 03:00:00+00,596402830,NaN,2020-11-20 14:45:06+00:00
16271,-76.994365,38.900203,20165859,2020/11/20 15:37:59+00,DAY,OTHERS,THEFT/OTHER,800 - 899 BLOCK OF H STREET NE,400489.0,136927.0,...,8402.0,Precinct 82,38.900195,-76.994363,NaN,2020/11/13 22:00:23+00,2020/11/14 00:00:13+00,596405482,NaN,2020-11-20 15:37:59+00:00
25170,-77.001316,38.898915,20165932,2020/11/20 18:56:18+00,DAY,OTHERS,THEFT F/AUTO,300 - 399 BLOCK OF G STREET NE,399886.0,136784.0,...,8301.0,Precinct 83,38.898907,-77.001314,NaN,2020/11/20 15:30:02+00,2020/11/20 18:25:35+00,596617921,NaN,2020-11-20 18:56:18+00:00


### 2.3 Use apply to cover all the other focal crimes

In [58]:
C_Tar.apply(find_related_crimes, axis = 1)

941                     X          Y       CCN         ...
14013                   X          Y       CCN         ...
dtype: object